In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter

torch.set_default_dtype(torch.float64)

In [26]:
# Tensorboard info from:
#
# https://pytorch.org/docs/stable/tensorboard.html
#

#
# Tensorboard needs to be called from the command line
#
#    tensorboard --logdir=runs
#

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [27]:
T=1000
STEP=int(T/10)

In [28]:
#
# a wrapper around nn.Module representing "Y = f(x)" that is being optimized
#
# in this case Y=X^2 and the loss function is X^2 as we're trying to minimize Y
#
# "guess" is the starting x-value
#
class Y(nn.Module):
    def __init__(self, guess):
        super().__init__()
        self.x = nn.Parameter(torch.tensor(guess))
        # this is the same:
        # self.x = nn.Parameter(torch.tensor(guess), requires_grad=True)
    
    def forward(self):
        return torch.pow(self.x,2)

    def loss(self):
        return torch.pow(self.x,2).item()

In [29]:
#
# initial guess = 1.0
#
model = Y(1.0)

optimizer = optim.SGD(model.parameters(), lr=0.1)
#x = torch.tensor(1.0, requires_grad=True)

for i in range(T+1):
    currApprox=model.x.item()
    writer.add_scalar("approx/step", currApprox, i)
    if i % STEP == 0:
        print(f"step={i}, approximation={currApprox}")
    optimizer.zero_grad()
    #e=model()
    e = model.forward()
    e.backward()
    optimizer.step()

writer.flush()

step=0, approximation=1.0
step=100, approximation=2.0370359763344877e-10
step=200, approximation=4.149515568880998e-20
step=300, approximation=8.452712498170662e-30
step=400, approximation=1.721847945638578e-39
step=500, approximation=3.507466211043411e-49
step=600, approximation=7.144834857673038e-59
step=700, approximation=1.4554285650048669e-68
step=800, approximation=2.9647603478997876e-78
step=900, approximation=6.039323489881814e-88
step=1000, approximation=1.2302319221611202e-97


In [30]:
import torch
from torch.autograd import Variable

init_val = torch.FloatTensor([1.0])
x = Variable(init_val, requires_grad=True)
#opt = torch.optim.Adam([x], lr=.1, betas=(0.5, 0.999))
opt = torch.optim.SGD([x], lr=.01)

for i in range(T+1):
    if i % int(STEP) == 0:
        print(f"step={i}, approximation={x.item()}")
    opt.zero_grad()
    curr_val = torch.pow(x,2)
    curr_val.backward() # Calculate gradients
    opt.step()

step=0, approximation=1.0
step=100, approximation=0.13261955976486206
step=200, approximation=0.017587954178452492
step=300, approximation=0.0023325064685195684
step=400, approximation=0.00030933599919080734
step=500, approximation=4.102399543626234e-05
step=600, approximation=5.4405822993430775e-06
step=700, approximation=7.215276127681136e-07
step=800, approximation=9.568864811626554e-08
step=900, approximation=1.2690186679265025e-08
step=1000, approximation=1.6829660998141094e-09


In [31]:
with SummaryWriter() as w:
    for i in range(5):
        w.add_hparams({'lr': 0.1*i, 'bsize': i, 'something':i**2},
                      {'accuracy': 10*i, 'loss': 10*i})
    w.flush()